Load modules

In [ ]:
from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.Attractor import base as attractor
from PacTimeOrig.utils import processing as proc
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from GLM import glm
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.diagnostics import hpdi

import patsy
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az

Load data, add in W_t, compute differential variables after reward re-alignment 

In [ ]:
#load datafile
datafile=dh.dataloader(sess=3,region='pMD',suffix='Pac_pMD.mat')

#Get session variables
sessvars=dh.getvars(datafile)

#Get position data
positions=dh.retrievepositions(datafile,rescale = [960.00,540.00])
psth=dh.get_psth(datafile,win_shift=30)
kinematics=dp.computederivatives(positions, vartodiff=['selfXpos','selfYpos','prey1Xpos','prey1Ypos','prey2Xpos','prey2Ypos'], dt=1.0/60.0)
# sessvars = dp.get_reaction_time(sessvars,kinematics)
sessvars = dh.get_matlab_wt_reaction_time(sessvars,session=3,subj='H')
#Select 2 prey trials
ogsessvars=sessvars
kinematics,sessvars =dh.subselect(kinematics,sessvars,trialtype='2')
psth,_= dh.subselect(psth,ogsessvars,trialtype='2')
#Drop columns
kinematics=dh.dropcols(kinematics, columns_to_drop=['predXpos','predYpos','prey1Xaccel','prey1Yaccel','prey2Xaccel','prey2Yaccel'])

#Get W_t vector
wtvec=dh.get_wt_vector(folder_path='/Users/user/PycharmProjects/PacManMain/data/WtNHP/H/NHP_H_SESSION_3/',selectype='average',transform=True)

#Cut data to correct length of wt
kinematics = dh.cut_to_rt(kinematics, sessvars)
# psth = [pd.DataFrame(x) for x in psth]
psth = dh.cut_to_rt(psth, sessvars)
kinematics = dh.get_time_vector(kinematics)
kinematics = dp.compute_distance(kinematics,trialtype=2)
#compute relative normalized speed
kinematics = dp.compute_relspeed(kinematics,trialtype=2)
kinematics = dp.compute_selfspeed(kinematics)

# Now merge wt:
wt = wtvec['wt']
wt=[pd.DataFrame(x) for x in wt]
wt=[df.rename(columns={0: 'wt'}) for df in wt]
kinematics = [df.merge(wt[i], left_index=True, right_index=True) for i, df in enumerate(kinematics)]

Compute relative value

In [ ]:
#For each kinematics frame, add relative reward value
Xdsgn = kinematics
for trial in range(len(Xdsgn)):
    Xdsgn[trial]['val1'] = np.repeat(sessvars.iloc[trial].NPCvalA,len(kinematics[trial]))
    Xdsgn[trial]['val2'] = np.repeat(sessvars.iloc[trial].NPCvalB,len(kinematics[trial]))
    
#Switch reward positions so highest value is always in prey 1 slot
Xdsgn=dh.rewardalign(Xdsgn)
Xdsgn = [df[sorted(df.columns)] for df in Xdsgn]

#Compute relative value
Xdsgn = [df.assign(relvalue=df['val1']-df['val2']).round(2) for df in Xdsgn]

Make data for design

In [ ]:
Xin = pd.concat(Xdsgn, axis=0, ignore_index=True)
Yin = pd.concat(psth, axis=0, ignore_index=True)
Xin['wt']+=0.1
Xin['wt']-=0.5

# Step 1: Perform linear regression to get slope and intercept
slope, intercept = np.polyfit(Xin['reldist'], Xin['wt'], 1)

# Step 2: Predict B using the linear model
B_pred = slope * Xin['reldist'] + intercept

# Step 3: Compute residuals
residuals = Xin['wt'] - B_pred

Xin['wt']=residuals

Formulas etc

In [ ]:
# y=Yin[4]
y=Yin[16]
formulas=[]
# # formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(reldist,df=5)+cr((wt),df=5))* C(relvalue)-1')
# # formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(reldist,df=5))* C(relvalue)-1')
# # formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(wt,df=5))* C(relvalue)-1')
# # formulas.append('y ~ (cr(reldist,df=5)+cr(wt,df=5))* C(relvalue)-1')
# formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(reldist,df=5)+cr((wt),df=5))-1')
# formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(reldist,df=5))-1')
# formulas.append('y ~ (cr(selfspeedmag,df=5)+cr(wt,df=5))-1')
# formulas.append('y ~ (cr(reldist,df=5)+cr(wt,df=5))-1')
# # formulas.append('y ~ (cr(selfspeedmag,df=5))* C(relvalue)-1')
# # formulas.append('y ~ (cr(reldist,df=5))* C(relvalue)-1')
# # formulas.append('y ~ (cr(wt,df=5))* C(relvalue)-1')
# formulas.append('y ~ cr(reldist,df=5)-1')
# formulas.append('y ~ cr(wt,df=5)-1')
# formulas.append('y ~ cr(selfspeedmag,df=5)-1')
# # formulas.append('y ~ C(relvalue)-1')
# formulas.append('y ~ te(cr(reldist,df=5),cr(wt,df=5))-1')

formulas.append('y ~ cr(wt,df=5)+cr(reldist,df=5)+cr(selfspeedmag,df=5)-1')
formulas.append('y ~ cr(wt,df=5)+cr(reldist,df=5)-1')
formulas.append('y ~ cr(wt,df=5)+cr(selfspeedmag,df=5)-1')
formulas.append('y ~ cr(reldist,df=5)+cr(selfspeedmag,df=5)-1')
formulas.append('y ~ cr(wt,df=5)-1')
formulas.append('y ~ cr(reldist,df=5)-1')
formulas.append('y ~ cr(selfspeedmag,df=5)-1')
formulas.append('y ~1')


mod=glm.PoissonGLM()
mod.add_data(Xin,y).make_preprocessor(formulas=formulas,metric='cv',l2reg=0.05).fit(params={'cv': 5, 'shuffleTime': False})

x1_range = np.linspace(np.median(jnp.array(Xin['selfspeedmag'])), np.median(np.array(Xin['selfspeedmag'])), 100)  # Example range for x1

x2_range=np.linspace(jnp.min(jnp.array(Xin['reldist'])), jnp.max(np.array(Xin['reldist'])), 100)  # Example range for x2
pred=[]
for _,x in enumerate(x2_range):
    x2_val = np.repeat(x, 100)  # Example range for x2
    x3_range = np.linspace(jnp.min(jnp.array(Xin['wt'])), jnp.max(jnp.array(Xin['wt'])), 100)  # Example range for x2
    data={"reldist": x2_val,"wt":x3_range}

    mod.predict(pd.DataFrame(data))
    pred.append(mod.predicted_y)

    Try Bayesian

In [ ]:


#Define formulw

basis_x = patsy.dmatrix("cr(Xin['selfspeedmag'],df=5)+cr(Xin['reldist'],df=5)", return_type="dataframe")


def model_Gaussian(basis_x, y=None):
    # Priors for coefficients
    coefs = numpyro.sample("coefs", dist.Normal(0, 0.01).expand([basis_x.shape[1]]))
    # Linear predictor using basis expansion
    linear_pred = jnp.dot(basis_x, coefs)
    # Poisson likelihood
    numpyro.sample("y", dist.Poisson(rate=jnp.exp(linear_pred)), obs=y)


def model_Laplace(basis_x, y=None):
    # Laplace prior for sparsity
    coefs = numpyro.sample("coefs", dist.Laplace(0, 1).expand([basis_x.shape[1]]))
    # Linear predictor using basis expansion
    linear_pred = jnp.dot(basis_x, coefs)
    # Poisson likelihood
    numpyro.sample("y", dist.Poisson(rate=jnp.exp(linear_pred)), obs=y)


nuts_kernel = NUTS(model_Gaussian)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=3000)
mcmc.run(jax.random.PRNGKey(0), basis_x=jnp.array(basis_x.values), y=jnp.array(y))
samples = mcmc.get_samples()


# Compute 90% credible intervals for each coefficient
credible_intervals = np.percentile(samples["coefs"], [5, 95], axis=0)
# Compute the mean estimate of each coefficient
mean_coefs = np.mean(samples["coefs"], axis=0)
# Zero out coefficients where the 95% CI overlaps with zer
adjusted_coefs = np.where((credible_intervals[0] <= 0) & (credible_intervals[1] >= 0), 0, mean_coefs)


x1_range = jnp.linspace(jnp.median(jnp.array(Xin['selfspeedmag'])), jnp.median(jnp.array(Xin['selfspeedmag'])), 100)  # Example range for x1
x2_range = jnp.linspace(jnp.min(jnp.array(Xin['reldist'])), jnp.max(jnp.array(Xin['reldist'])), 100)  # Example range for x2
data = {"selfspeedmag": x1_range, "reldist": x2_range}


basis_x = patsy.dmatrix("cr(data['selfspeedmag'],df=5)+cr(data['reldist'],df=5)", return_type="dataframe")


basis_x = jnp.array(basis_x.values)  # Convert to JAX array

# Compute predictions for each posterior sample
linear_preds_samples = jnp.dot(samples["coefs"], basis_x.T)
rate_preds_samples = jnp.exp(linear_preds_samples)  # Poisson rate (lambda)

# Compute mean prediction and HPDI for each x1 value
mean_rate_pred = rate_preds_samples.mean(axis=0)
hpdi_rate_pred = hpdi(rate_preds_samples, prob=0.95) 